In [12]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor  
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Load the datasets
census_data = pd.read_csv('Census total.csv')
district_data = pd.read_csv('updated_districts_cleaned.csv')

# Process 'Census Tract' column and ensure 'year' is an integer
census_data['Census Tract'] = census_data['Census Tract'].str.extract('(\d+\.?\d*)')[0]
census_data['year'] = pd.to_numeric(census_data['year'], errors='coerce').astype('Int64')
district_data['Census Tract'] = district_data['Census Tract'].astype(str)

# Merge the data
combined_data = pd.merge(census_data, district_data, on='Census Tract', how='inner')

# Convert 'Median income (dollars)' to numeric
combined_data['Median income (dollars)'] = pd.to_numeric(combined_data['Median income (dollars)'], errors='coerce')

# Define and process all race columns
race_columns = ['White alone', 'Black or African American alone', 'Asian alone', 
                'American Indian and Alaska Native alone', 'Native Hawaiian and Other Pacific Islander alone', 
                'Some other race alone', 'Two or more races']
for col in race_columns:
    combined_data[col] = pd.to_numeric(combined_data[col], errors='coerce').fillna(0)

# Define age midpoints and calculate average age
age_midpoints = {
    'Under 5 years': 2.5, '5 to 9 years': 7.5, '10 to 14 years': 12.5, '15 to 19 years': 17.5, 
    '20 to 24 years': 22.5, '25 to 29 years': 27.5, '30 to 34 years': 32.5, '35 to 39 years': 37.5, 
    '40 to 44 years': 42.5, '45 to 49 years': 47.5, '50 to 54 years': 52.5, '55 to 59 years': 57.5, 
    '60 to 64 years': 62.5, '65 to 69 years': 67.5, '70 to 74 years': 72.5, '75 to 79 years': 77.5, 
    '80 to 84 years': 82.5
}

for age_group in age_midpoints:
    if age_group in combined_data.columns:
        combined_data[age_group] = pd.to_numeric(combined_data[age_group], errors='coerce').fillna(0)

combined_data['Total years'] = sum(combined_data[col] * age_midpoints[col] for col in age_midpoints if col in combined_data.columns)
combined_data['Total Population'] = combined_data[[col for col in age_midpoints if col in combined_data.columns]].sum(axis=1)
combined_data['Average Age'] = combined_data['Total years'] / combined_data['Total Population']

features = combined_data[['Median income (dollars)', 'Average Age'] + race_columns]  
target = combined_data['Average Age'] 

# Handling missing values if necessary
features.fillna(features.mean(), inplace=True)
target.fillna(method='ffill', inplace=True)  # or another method as appropriate

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Create a Random Forest regression model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean Squared Error:", mse)
print("R-squared:", r2)

# Further analysis like feature importance, hyperparameter tuning, etc., can be added as per requirement.


C:\Users\Jcp45\AppData\Local\Temp\ipykernel_20640\1781846648.py:8: DtypeWarning: Columns (1,2,32,33,34,35,36,37,38,39,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  census_data = pd.read_csv('Census total.csv')
C:\Users\Jcp45\AppData\Local\Temp\ipykernel_20640\1781846648.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.fillna(features.mean(), inplace=True)
C:\Users\Jcp45\AppData\Local\Temp\ipykernel_20640\1781846648.py:51: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  target.fillna(method='ffill', inplace=True)  # or another method as appropriate


Mean Squared Error: 5.453335766255624e-26
R-squared: 1.0
